In [13]:
val file = spark.read.textFile("soc-LiveJournal1.txt").rdd
var header = file.first()
var lines = file.filter(row => row != header)
val pairs = lines.map{ s =>
      val parts = s.split("\\s+")               // Splits a line into an array of 2 elements according space(s)
             (parts(0), parts(1))                 // create the parts<url, url> for each line in the file
    }
val links = pairs.distinct().groupByKey()

file: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[752] at rdd at <console>:29
header: String = # FromNodeId	ToNodeId
lines: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[753] at filter at <console>:31
pairs: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[754] at map at <console>:32
links: org.apache.spark.rdd.RDD[(String, Iterable[String])] = ShuffledRDD[758] at groupByKey at <console>:36


In [14]:
var ranks = links.mapValues(v => 1.0)
for (i <- 1 to 50) {
    val contribs = links.join(ranks)         // join  -> RDD1
         .values                           // extract values from RDD1 -> RDD2          
         .flatMap{ case (urls, rank) =>    // RDD2 -> conbrib RDD
                 val size = urls.size        
                     urls.map(url => (url, rank / size))   // the ranks are distributed equally amongs the various URLs
             }
    ranks = contribs.reduceByKey(_ + _).mapValues(0.05 + 0.95 * _) // ranks RDD
}

ranks: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[1109] at mapValues at <console>:38


In [15]:
var sorted = ranks.sortBy({ case (_, rank) => rank }, ascending = false)
sorted.take(100).foreach(println)

(13948,4.242923673281286)
(39394,4.1466372490805385)
(2678,3.502783386354853)
(1860,3.3834146012685316)
(31579,3.116876831018664)
(65688,3.061540502768572)
(28291,2.7125961746490015)
(47507,2.664186143747885)
(8092,2.561296081575024)
(649,2.53610738547489)
(65674,2.5256481815174743)
(65677,2.4392377825956655)
(2914,2.388312350661569)
(17074,2.367575664302596)
(134,2.311135952826648)
(17604,2.2739022948645644)
(12241,2.155953783925255)
(12665,2.1168109889580613)
(3240,2.094133801978064)
(1694,2.0772926641427247)
(59389,2.055839591992915)
(3894,2.053954920531637)
(30305,2.031345763102963)
(91195,1.9710997923694995)
(47494,1.9633175173087263)
(22165,1.9145055820176622)
(89076,1.9107406038592472)
(8737,1.9008589492545938)
(37847,1.8944593050582892)
(38906,1.791790185363488)
(2659,1.767425802576682)
(42698,1.7228830336607208)
(1854,1.646857657809781)
(19998,1.6427244664749094)
(15761,1.6382626125022899)
(35089,1.600525976921303)
(1220,1.5867346768397104)
(27909,1.5833299932660472)
(214538,1

sorted: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[1114] at sortBy at <console>:28
